In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

from datetime import datetime
from dateutil.relativedelta import relativedelta

from empresa4.datasets import nombres_datasets,get_dataset

In [3]:
#df q usamos de ejemplo para todo el procesamiento q se hace abajo
df = get_dataset("01_productos_todos")
#df

# Lags

In [4]:
def dame_lags_y_deltas(df,nombre_col_original,cant_lags,incluir_deltas=False):
    """
    Agregar columnas con lags y (opcionalmente) deltas
    
    Input:
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - cant_lags: (int) Entero que define la cantidad de lags q va a calcular
    - incluir_deltas: (bool) Se usa para definir si se van a calcular los deltas (True=calcula deltas)
 
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
 
    Ejemplo:
    df = dame_lags_y_deltas(df,nombre_col_original="tn",cant_lags=12,incluir_deltas=True)

    """
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    #Calculamos los lags
    nombres_columnas_lags = []
    for i in range(1,cant_lags+1):
        nombre_nueva_columna = nombre_col_original+"_lag_"+str(i)
        nombres_columnas_lags.append(nombre_nueva_columna)
        df_aux = df.groupby(['product_id'])[nombre_col_original].shift(i).rename(nombre_nueva_columna)
        df=pd.concat([df,df_aux],axis=1)  
        
    #Si corresponde, calculamos los delta lags
    if incluir_deltas == True:
        prefijo_delta_lags = "delta_lag_" # va a tener ESTO mas un numero de 1 a N
        df_aux=pd.DataFrame()
        #Se calculan lag1-lag2, lag2-lag3, lag3-lag4, etc...
        for i in range(0,len(nombres_columnas_lags)-1):
            df_aux[prefijo_delta_lags+str(i+1)] = df[nombres_columnas_lags[i]]-df[nombres_columnas_lags[i+1]]
        #Lo unificamos con el df original
        df = pd.concat([df,df_aux],axis=1)  
        
    return df.copy()

In [5]:
nombre_col_original = "tn" # columna sobre la cual queremos calcular los lags
cant_lags = 12

df = dame_lags_y_deltas(df,nombre_col_original,cant_lags,incluir_deltas=True)

In [6]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None):
#     display(df.head(50))

# Medias moviles



In [7]:
def dame_medias_moviles(df,nombre_col_original,lista_medias_target,incluye_periodo_actual=False):
    
    """
    Calculamos las medias de los ultimos N meses.
    Cada entero "k" que agregues al vector de aca abajo va a crear una columna nueva con 
    el promedio de esos "k" meses previos 
    Podes controlar si incluye o no al periodo actual mediante la variable incluye_periodo_actual
    
    Input: 
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - lista_medias_target: (list) Lista de enteros para saber el tamanio de las ventanas para las medias moviles
    - incluye_periodo_actual: (bool) Es para definir si se debe tener en cuenta o no el periodo actual para el calculo de la media movil (por defecto es False, asi que NO se usa el periodo actual)
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo:
    df = dame_medias_moviles(df,nombre_col_original="tn",lista_medias_target=[1,3,6,12],incluye_periodo_actual=False)
    
    """
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    prefijo = nombre_col_original + "_media_movil_"
    
    if incluye_periodo_actual==True:
        SHIFT=0
    else:
        SHIFT=1

    for i in lista_medias_target:
        df_aux = df.groupby('product_id').rolling(window=i,min_periods=i)[nombre_col_original].mean().shift(SHIFT).rename(prefijo+str(i)).reset_index(drop=True)
        df = pd.concat([df,df_aux],axis=1)
        
    return df.copy()

In [8]:
lista_medias_target = [2,3,4,6,10] # cant periodos, en meses, que debe medir de largo la ventana
incluye_periodo_actual = False  # definir si la ventana debe incluir o no al periodo actual
nombre_col_original = "tn"

df = dame_medias_moviles(df,nombre_col_original,lista_medias_target,incluye_periodo_actual)

In [9]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))

# Tendencias



In [10]:
def dame_tendencias(df,nombre_col_original,lista_tamanio_ventana_tendencias,incluye_periodo_actual=False):
    """
    Calcula la pendiente de la recta q aproxima a los elementos dentro de una ventana movil de periodos.
   
   Input:
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - lista_tamanio_ventana_tendencias: (list) Lista de enteros para indicar cant de periodos, en meses, que debe medir de largo la ventana
    - incluye_periodo_actual: (bool) Es para definir si la ventana debe incluir o no al periodo actual. Por defecto vale False (asi q NO lo incluye)

    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo:
    df = dame_tendencias(df,nombre_col_original="tn", lista_tamanio_ventana_tendencias=[3,6,12],incluye_periodo_actual=False)
    """
    
    prefijo = nombre_col_original + "_tendencia_"

    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    if incluye_periodo_actual==True:
        SHIFT=0
    else:
        SHIFT=1

    for i in lista_tamanio_ventana_tendencias:
        df_aux = df.groupby('product_id')[nombre_col_original].rolling(window=i, min_periods=i).apply(lambda v: linregress(np.arange(len(v)), v).slope).shift(SHIFT).rename(prefijo+str(i)).reset_index(level=0, drop=True)
        df = pd.concat([df,df_aux],axis=1)
        
    return df.copy()

In [11]:
lista_tamanio_ventana_tendencias = [3,6,12] # cant de periodos, en meses, que debe medir de largo la ventana
incluye_periodo_actual = False # definir si la ventana debe incluir o no al periodo actual
nombre_col_original="tn"

df = dame_tendencias(df,nombre_col_original, lista_tamanio_ventana_tendencias,incluye_periodo_actual)

In [12]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))

# Ventas N meses atras



In [13]:
def dame_ventas_n_meses_atras(df,nombre_col_original,lista_ventas_n_meses_atras):
    """
    Calculo de las ventas realizadas hace N meses atras  
    
    Input:
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - lista_ventas_n_meses_atras: (list) Lista de enteros donde especificamos cuales meses hacia atras queremos tener como referencia
   
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo:
    df = dame_ventas_n_meses_atras(df,nombre_col_original="tn",lista_ventas_n_meses_atras=[1,2,4,8])
    """
    
    prefijo = nombre_col_original + "_venta_pasado_" #Para modificar el prefijo de la columna nueva
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    for i in lista_ventas_n_meses_atras:
        df_aux = df.groupby("product_id")[nombre_col_original].shift(i).rename(prefijo+str(i))
        #display(df_aux)
        df = pd.concat([df,df_aux],axis=1)
        
    return df.copy()

In [14]:
lista_ventas_n_meses_atras = [1,2,5,10] #Especificamos a cuales meses hacia atras queremos tener como referencia

df = dame_ventas_n_meses_atras(df,nombre_col_original,lista_ventas_n_meses_atras)

In [15]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))
#     #display(df.tail(50))

### Estacionalidad: nro de MES de cada registro

In [16]:
def dame_estacionalidad(df):

    """
    Inserta en el df una columna q indica el mes del anio al que corresponde el registro
    
    Input:
    - df: (dataframe) Debe contener el campo "periodo".
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo: 
    df = dame_estacionalidad(df)
    """
    
    nombre_nueva_columna = "mes_del_anio"
    df[nombre_nueva_columna] = (df["periodo"]%100)

    return df.copy()


In [17]:
df = dame_estacionalidad(df)

In [18]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))
#     #display(df.tail(50))

### Tn de los productos de igual marca, pero tamanio inmediatamente mayor y menor

In [19]:
def dame_hermanos_mayores_y_menores(df, nombre_col_original, periodos_previos):

    """
    Requiere tener instalada la biblioteca empresa4.
    
    Se genera información de ventas (tn) de productos de igual categoria 1,2 y 3 e igual marca, pero distinto tamanio.
    Llamamos Hermano Mayor a un producto de este tipo, cuando es mas grande que otro.
    Llamamos Hermano Menor a un producto de este tipo, cuando es mas chico que otro.
    
    Esta funcion permite incorporar columnas con info de ventas de los hermanos mayores y menores para N periodos en el pasado.
    
    Input:
    * df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    * nombre_col_original: (string) Es la columna de referencia (por ejemplo "tn")
    * periodos_previos: (list) Es una lista de enteros para indicar los periodos sobre los que quiero traer info de ventas de hermanos mayores y menores
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas de toneladas vendidas, pero tambien el product_id de los hermanos mayor y menor
                      Nota: "hermano_M" se refiere a "hermano mayor" mientras que "hermano_m" se refiere a "hermano menor".
                    
    Ejemplo:
    df = dame_hermanos_mayores_y_menores(df,nombre_col_original="tn",periodos_previos=[1,2,5])
    
    """    
    
    # En este df tenemostodos los productos (product_id), 
    # con las categorias a las q pertenecen (de nivel 1, 2 y 3)
    df_productos = get_dataset("maestro_productos")

    #Agrupamos por cat1,cat2 y cat3 y brand
    df_productos.sort_values(by=["cat1","cat2","cat3","brand","sku_size"],inplace=True)
    grupos = df_productos.groupby(["cat1","cat2","cat3","brand"])[["product_id"]]

    df_prod_hermanos = pd.DataFrame()
    for grupo in grupos:
        df_aux = grupo[1].copy()
        df_aux ["hermano_mayor"] = grupo[1].shift(-1)
        df_aux ["hermano_menor"] = grupo[1].shift(1)
        df_prod_hermanos = pd.concat([df_prod_hermanos,df_aux])
    # #en este df tenemos cada product_id con info de quienes son los productos hermano_mayor y hermano_menor
    # df_prod_hermanos
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)

    #le agregamos a df la info de quien es hermano_maryo y menor de cada product_id
    df=pd.merge(
        df, #left
        df_prod_hermanos, #right
        how="left",
        on="product_id",
    )

    # Ahora haremos dos cosas:
    # 1) Agarrar un product id en un periodo -> fijate quien es el hno mayor, traeme lo q vendio en N periodos del pasado (sin incluir el actual)
    # 2) Agarrar un product id en un periodo -> fijate quien es el hno menor, traeme lo q vendio en N periodos del pasado (sin incluir el actual)

    acumulado_hermanos_menores = []
    acumulado_hermanos_mayores = []

    for idx,row in df.iterrows():

        periodo_actual = row["periodo"]
        hermano_menor = row["hermano_menor"]
        hermano_mayor = row["hermano_mayor"]
        hermano_del_medio = row["product_id"]

        #Levantamos la info de todos los hermanos menores
        if np.isnan(hermano_menor)==False:
            df_aux = df.loc[ df["product_id"] == hermano_menor ]
            acumulado_aux = []
            for i in periodos_previos:
                #convertimos a date, restamos un dia, y volvemos a int:
                periodo_target = pd.to_datetime(str(periodo_actual), format='%Y%m') - relativedelta(months=i)
                periodo_target = int(periodo_target.strftime('%Y%m'))
                aux = df_aux.loc [df_aux["periodo"]==periodo_target][nombre_col_original].to_list()
                if len(aux)==0:
                    aux = [np.NaN]
                acumulado_aux.append(aux)
            acumulado_aux = [n for elemento in acumulado_aux for n in elemento]
        else:
            #Si entra aca, es porque NO tiene hermanos menores
            #asi que las tn de los hermanos menores es todo NAN
            acumulado_aux = [np.nan for j in range(0,len(periodos_previos))]

        acumulado_hermanos_menores.append(acumulado_aux)

        #generamos la info de todos los hermanos mayores
        if np.isnan(hermano_mayor)==False:
            df_aux = df.loc[ df["product_id"] == hermano_mayor ]
            acumulado_aux = []
            for i in periodos_previos:
                #convertimos a date, restamos un dia, y volvemos a int:
                periodo_target = pd.to_datetime(str(periodo_actual), format='%Y%m') - relativedelta(months=i)
                periodo_target = int(periodo_target.strftime('%Y%m'))
                aux = df_aux.loc [df_aux["periodo"]==periodo_target][nombre_col_original].to_list()
                if len(aux)==0:
                    aux = [np.NaN]
                acumulado_aux.append(aux)
            acumulado_aux = [n for elemento in acumulado_aux for n in elemento]
        else:
            #Si entra aca, es porque NO tiene hermanos mayores
            #asi que las tn de los hermanos mayores es todo NAN
            acumulado_aux = [np.nan for j in range(0,len(periodos_previos))]

        acumulado_hermanos_mayores.append(acumulado_aux)    

    #Finalmente, armamos un df con toda la info
    df = pd.concat(
        [
        df,
        pd.DataFrame(acumulado_hermanos_mayores, columns = [nombre_col_original+"_hermano_M_-"+str(periodos_previos[i])   for i in range(0,len(periodos_previos))    ]), 
        pd.DataFrame(acumulado_hermanos_menores, columns = [nombre_col_original+"_hermano_m_-"+str(periodos_previos[i])   for i in range(0,len(periodos_previos))    ])
        ], axis=1
    )
    
    return df.copy()


In [20]:
periodos_previos = [1,2,5] #periodos sobre los que quiero traer info de ventas de hermanos mayores y menores
nombre_col_original = "tn"

df = dame_hermanos_mayores_y_menores(df,nombre_col_original,periodos_previos)

In [21]:
#Vemos como quedo todo
with pd.option_context('display.max_columns', None): 
    display(df.head(50))

,product_id,periodo,cust_request_qty,cust_request_tn,tn,product_category,tn_lag_1,tn_lag_2,tn_lag_3,tn_lag_4,tn_lag_5,tn_lag_6,tn_lag_7,tn_lag_8,tn_lag_9,tn_lag_10,tn_lag_11,tn_lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_movil_2,tn_media_movil_3,tn_media_movil_4,tn_media_movil_6,tn_media_movil_10,tn_tendencia_3,tn_tendencia_6,tn_tendencia_12,tn_venta_pasado_1,tn_venta_pasado_2,tn_venta_pasado_5,tn_venta_pasado_10,mes_del_anio,hermano_mayor,hermano_menor,tn_hermano_M_-1,tn_hermano_M_-2,tn_hermano_M_-5,tn_hermano_m_-1,tn_hermano_m_-2,tn_hermano_m_-5
0,20001,201701,479.0,937.72717,934.77222,HC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20370.0,20102.0,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,432.0,833.72187,798.01620,HC,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,934.77222,NaN,NaN,NaN,2,20370.0,20102.0,0.00000,NaN,NaN,116.86614,NaN,NaN
2,20001,201703,509.0,1330.74697,1303.35771,HC,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-136.75602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866.394210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798.01620,934.77222,NaN,NaN,3,20370.0,20102.0,0.00000,0.00000,NaN,68.92650,116.86614,NaN
3,20001,201704,279.0,1132.94430,1069.96130,HC,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,505.34151,-136.75602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050.686955,1012.048710,NaN,NaN,NaN,184.292745,NaN,NaN,1303.35771,798.01620,NaN,NaN,4,20370.0,20102.0,0.00000,0.00000,NaN,159.98159,68.92650,NaN
4,20001,201705,701.0,1550.68936,1502.20132,HC,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1186.659505,1057.111737,1026.526858,NaN,NaN,135.972550,NaN,NaN,1069.96130,1303.35771,NaN,NaN,5,20370.0,20102.0,14.47628,0.00000,NaN,77.07501,159.98159,NaN
5,20001,201706,570.0,1575.82891,1520.06539,HC,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,432.24002,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1286.081310,1291.840110,1168.384133,NaN,NaN,99.421805,NaN,NaN,1502.20132,1069.96130,934.77222,NaN,6,20370.0,20102.0,23.74408,14.47628,0.00000,125.65265,77.07501,116.86614
6,20001,201707,381.0,1086.47101,1030.67391,HC,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,17.86407,432.24002,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,NaN,NaN,NaN,1511.133355,1364.076003,1348.896430,1188.062357,NaN,225.052045,137.303566,NaN,1520.06539,1502.20132,798.01620,NaN,7,20370.0,20102.0,33.41482,23.74408,0.00000,101.58766,125.65265,68.92650
7,20001,201708,643.0,1289.66869,1267.39462,HC,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,-489.39148,17.86407,432.24002,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,NaN,NaN,1275.369650,1350.980207,1280.725480,1204.045972,NaN,-235.763705,64.161475,NaN,1030.67391,1520.06539,1303.35771,NaN,8,20370.0,20102.0,27.63924,33.41482,0.00000,86.84650,101.58766,159.98159
8,20001,201709,381.0,1356.96103,1316.94604,HC,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,236.72071,-489.39148,17.86407,432.24002,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,NaN,1149.034265,1272.711307,1330.083810,1282.275708,NaN,-126.335385,-7.994673,NaN,1267.39462,1030.67391,1069.96130,NaN,9,20370.0,20102.0,20.92346,27.63924,14.47628,76.86229,86.84650,77.07501
9,20001,201710,273.0,1441.60247,1439.75563,HC,1316.94604,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,49.55142,236.72071,-489.39148,17.86407,432.24002,-233.39641,505.34151,-136.75602,NaN,NaN,NaN,1292.170330,1205.004857,1283.769990,1284.540430,NaN,143.136065,1.174632,NaN,1316.

In [25]:
#Para hacer un Chequeo a ver si estan bien armados los datos de los hermanos Mayor y menor: 
#df.loc[ df["product_id"]==20370.0] #hermano Mayor

In [26]:
#df.loc[ df["product_id"]==20001] #hermano del medio

In [27]:
#df.loc[ df["product_id"]==20102.0] #hermano menor

In [ ]:
#Guardamos el archivo de salida junto con todos los datasets
#df.to_csv("output_FE.csv",index=False)